## virus host links
- Link vOTUs to putative hosts
- Predict CRISPR-spacer regions using minced, https://github.com/ctSkennerton/minced
- Deduplicate at 100% ANI 
- Blast against vOTUs (own), vOTUs (RVD)
- Get linkages
- If linked, check abundance of host and virus through samples using either manysearch of fmg


blast options:
and matches with ≤ 1 mismatch and an E-value ≤ 1e−5 
https://www.nature.com/articles/s41467-023-38400-0

CRISPR spacers in genome bins were BLASTn55 searched against all viral elements, with hits retained only if they had up to one mismatch, a query coverage of ≥ 90% and an E-value ≤ 10−4
https://www.nature.com/articles/s41598-023-32078-6

In [ ]:
# bins  (n=17558), *.fa
/home/hfm/ER_rumenShotgun/ER_Nanuq/MAG/bins 

In [ ]:
# Running both minced and sourmash to sketch the bins so I can get some taxonomy
# snakemake virsorter on the subsetted files
srun --account=ctbrowngrp -p bmh -J minced -t 20:00:00 -c 80 --mem=50gb --pty bash
srun --account=ctbrowngrp -p med2 -J minced -t 24:00:00 -c 30 --mem=50gb --pty bash


mamba activate branchwater
snakemake --use-conda --resources mem_mb=50000 --rerun-triggers mtime \
-c 30 --rerun-incomplete -k -s Snakefile_crispr


In [ ]:
# fastmultigather
srun --account=ctbrowngrp -p bmh -J fmg -t 12:00:00 -c 32 --mem=100gb --pty bash

# Do fastmultigather on all sketches
cd ../results/sourmash/MAGsketch
readlink -f *.zip > ../MAG_sketch.filepath.txt

# do fastmultigather against genbank
/group/ctbrowngrp/sourmash-db/genbank-2022.03/genbank-2022.03-bacteria.lineages.csv.gz
/group/ctbrowngrp/sourmash-db/genbank-2022.03/genbank-2022.03-archaea.lineages.csv.gz
/group/ctbrowngrp/sourmash-db/genbank-2022.03/genbank-2022.03-protozoa.lineages.csv.gz


# fmg bacteria
sourmash scripts fastmultigather \
../MAGsketch.filepath.txt \
/group/ctbrowngrp/sourmash-db/genbank-2022.03/genbank-2022.03-bacteria-k31.zip \
-k 31 --scaled 1000 -m DNA -c 32

# fmg archeae
sourmash scripts fastmultigather \
../MAGsketch.filepath.txt \
/group/ctbrowngrp/sourmash-db/genbank-2022.03/genbank-2022.03-archaea-k31.zip \
-k 31 --scaled 1000 -m DNA -c 32

# protozoa
sourmash scripts fastmultigather \
../MAGsketch.filepath.txt \
/group/ctbrowngrp/sourmash-db/genbank-2022.03/genbank-2022.03-protozoa-k31.zip \
-k 31 --scaled 1000 -m DNA -c 32

In [ ]:
# tax classify
readlink -f *.gather.csv > ../tax/archaea.txt

In [ ]:
# tax classify them 
sourmash tax genome -q --from-file protozoa.txt \
-t /group/ctbrowngrp/sourmash-db/genbank-2022.03/genbank-2022.03-protozoa.lineages.csv.gz \
--output-dir protozoa -o protozoa

sourmash tax genome -q --from-file bacteria.txt \
-t /group/ctbrowngrp/sourmash-db/genbank-2022.03/genbank-2022.03-bacteria.lineages.csv.gz \
--output-dir . -o bacteria

In [ ]:
# go to the crispr folder
cd ./results/crispr/

# concatenate all spacers (n=32328)
cat *.fa > ../all_spacers.fa

# Cat rvd and own contigs (all, not deduplicated)
cat RVD_rn.fa viral_contigs.rn.fa > RVD_owncontigs.fa

# make a blastdb of all vOTU sequences (no dereplication, may be interesting strain diversity))
srun --account=ctbrowngrp -p high2 -J blastn -t 12:00:00 -c 8 --mem=50gb --pty bash

# make a db 
mamba activate blast
makeblastdb -in RVD_owncontigs.fa -dbtype nucl -out ./blastdb/RVD_owncontigs

# Blast spacers to vOTUs (83,409 linkages).
# Extract RVD links and do coverM for these too
blastn -task blastn-short -num_threads 8 \
-query all_spacers.minced.fa \
-db ./blastdb/RVD_owncontigs \
-evalue 1e-5 -perc_identity 95 -outfmt 6 \
-out spacers_vOTUs.rumen.minced 

In [ ]:
# then concat those rumen virs with our virs, deduplicate and make a coverage table. 
# Maybe even all rumen ones?
